In [6]:
#Extract Notes and Piano_rolls from MIDI Files

import os
import numpy as np
import pandas as pd
import pretty_midi
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import glob

# Define the dataset path
dataset_path = "/content/drive/MyDrive/DL Lab - 2nd Sem/Lab 5/Dataset/maestro-v3.0.0/"

midi_files = glob.glob(os.path.join(dataset_path, "**/*.midi"), recursive=True)
midi_files += glob.glob(os.path.join(dataset_path, "**/*.mid"), recursive=True)

# Function to extract note sequences from MIDI
def midi_to_notes(midi_file):
    pm = pretty_midi.PrettyMIDI(midi_file)
    notes = []

    for instrument in pm.instruments:
        if instrument.is_drum:
            continue  # Skip drum tracks

        for note in instrument.notes:
            notes.append([note.start, note.end, note.pitch, note.velocity])

    return np.array(notes)

# Process all MIDI files
all_notes = [midi_to_notes(f) for f in midi_files]
all_notes = np.concatenate(all_notes, axis=0)

# Exploratory Data Analysis (EDA) on MIDI Notes
plt.figure(figsize=(10, 4))
sns.histplot(all_notes[:, 2], bins=50, kde=True, color="blue")  # MIDI pitch distribution
plt.xlabel("MIDI Pitch")
plt.ylabel("Frequency")
plt.title("Pitch Distribution in MAESTRO Dataset")
plt.show()

# Function to convert MIDI to piano roll
def midi_to_piano_roll(midi_file, fs=100):
    pm = pretty_midi.PrettyMIDI(midi_file)
    piano_roll = pm.get_piano_roll(fs=fs)  # fs = time steps per second
    return piano_roll.T  # Transpose to have (time_steps, pitch_classes)

# Convert all MIDI files to piano rolls
piano_rolls = [midi_to_piano_roll(f) for f in midi_files]

# Define a fixed maximum sequence length
max_length = 1000

# Pad or truncate sequences to the same length
piano_rolls = [x[:max_length] if x.shape[0] > max_length else np.pad(x, ((0, max_length - x.shape[0]), (0, 0))) for x in piano_rolls]

# Convert to NumPy array
X = np.array(piano_rolls)
max = np.max(X)
print(max)

# Train-Test-Validation Split
X_train, X_temp = train_test_split(X, test_size=0.2, random_state=42)
X_val, X_test = train_test_split(X_temp, test_size=0.5, random_state=42)

# Normalize input data
X_train = X_train / max  # Normalize MIDI velocities
X_val = X_val / max
X_test = X_test / max

# Print data shapes
print(f"Training Data Shape: {X_train.shape}")
print(f"Validation Data Shape: {X_val.shape}")
print(f"Test Data Shape: {X_test.shape}")

ValueError: need at least one array to concatenate

In [2]:
!pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.3 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=ea7e74d97fbeb30d250c30c222ee492b83dac075f7fed7a0200a4df72af44e1b
  Stored in directory: /root/.cache/pip/wheels/e6/95/ac/15ceaeb2823b04d8e638fd1495357adb8d26c00ccac9d7782e
Successfully built pretty_midi
